<a href="https://colab.research.google.com/github/kevinsconnolly/Big_Data/blob/master/OfficeProductsChallengeDeliverable.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.0'
spark_version = 'spark-3.0.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://security.ubuntu.com/ubuntu bionic-security InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:9 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic InRelease
Hit:10 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:13 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease
Reading package lists... Done


In [ ]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2020-09-20 23:10:16--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar.7’

postgresql-42.2.16. 100%[===================>] 979.38K  --.-KB/s    in 0.1s    

2020-09-20 23:10:16 (9.12 MB/s) - ‘postgresql-42.2.16.jar.7’ saved [1002883/1002883]



In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [ ]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url ="https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Office_Products_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Office_Products_v1_00.tsv.gz"), sep="\t", header=True)

In [ ]:
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   43081963|R18RVCKGH1SSI9|B001BM2MAC|     307809868|Scotch Cushion Wr...| Office Products|          5|            0|          0|   N|                Y|          Five Stars|      Great product.| 2015-08-31|
|         US|   10951564|R3L4L6LW1PUOFY|B00DZYEXPQ|      75004341|Dust-Off Compress...| Office Products|          5|    

In [ ]:
df.count()

2642434

In [ ]:
# Drop any rows with either null or “not a number” (NaN) values
office_df = df.dropna()

In [ ]:
office_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   43081963|R18RVCKGH1SSI9|B001BM2MAC|     307809868|Scotch Cushion Wr...| Office Products|          5|            0|          0|   N|                Y|          Five Stars|      Great product.| 2015-08-31|
|         US|   10951564|R3L4L6LW1PUOFY|B00DZYEXPQ|      75004341|Dust-Off Compress...| Office Products|          5|    

In [ ]:
office_df.count()

2642195

In [ ]:
Yes_df = office_df.filter(office_df["vine"] == "Y")

In [ ]:
Yes_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   14220514|R2W2C9AQHPK31R|B00TKU1NHA|      77272979|Westcott iPoint O...| Office Products|          5|            0|          0|   Y|                N|Kid proof, or as ...|It seems electric...| 2015-08-31|
|         US|   53093124|R2DCXYKMNALS1O|B00UV41MAC|      85017495|V-LIGHT LED Energ...| Office Products|          5|    

In [ ]:
Yes_df.count()

29186

In [ ]:
No_df = office_df.filter(office_df["vine"] =="N")

In [ ]:
No_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   43081963|R18RVCKGH1SSI9|B001BM2MAC|     307809868|Scotch Cushion Wr...| Office Products|          5|            0|          0|   N|                Y|          Five Stars|      Great product.| 2015-08-31|
|         US|   10951564|R3L4L6LW1PUOFY|B00DZYEXPQ|      75004341|Dust-Off Compress...| Office Products|          5|    

In [ ]:
No_df.count()

2613009

In [ ]:
marketplace_df = office_df.filter(office_df["marketplace"] == "US")

In [ ]:
marketplace_df.count()

2642195

In [ ]:
filtered1_df = df.select("customer_id", "product_parent", "product_title", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase", "review_headline", "review_body", "review_date").show()

+-----------+--------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|customer_id|product_parent|       product_title|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+--------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|   43081963|     307809868|Scotch Cushion Wr...|          5|            0|          0|   N|                Y|          Five Stars|      Great product.| 2015-08-31|
|   10951564|      75004341|Dust-Off Compress...|          5|            0|          1|   N|                Y|Phffffffft, Phfff...|What's to say abo...| 2015-08-31|
|   21143145|     529689027|Amram Tagger Stan...|          5|            0|          0|   N|                Y|but I am sure I w...|Haven't used yet,...| 2015-08-31|
|   527823

In [ ]:
review_date_df = office_df.filter(office_df["review_date"] == "2015-08-31")

In [ ]:
review_date_df.count()

4157

In [ ]:
#From the date value of 2015-08-31 has 4,157 rows in our 2,642,195 row data frame. This shows our review_date columns holds multiple dates and has no substantical value in our analysis


In [ ]:
filtered2_df = df.select("customer_id", "product_parent", "product_title", "vine", "review_headline", "review_body")

In [ ]:
filtered2_Y_df = filtered2_df.filter(filtered2_df["vine"] == "Y")

In [ ]:
filtered2_Y_df.show()

+-----------+--------------+--------------------+----+--------------------+--------------------+
|customer_id|product_parent|       product_title|vine|     review_headline|         review_body|
+-----------+--------------+--------------------+----+--------------------+--------------------+
|   14220514|      77272979|Westcott iPoint O...|   Y|Kid proof, or as ...|It seems electric...|
|   53093124|      85017495|V-LIGHT LED Energ...|   Y|... V-Light LED E...|The V-Light LED E...|
|   43079307|     646830004|Elmer's Freestyle...|   Y|              Useful|I use this light ...|
|   46278244|     892837487|Boxa HOPPER Elega...|   Y|Not exciting, but...|This is a review ...|
|   52856050|     338829902|Elmer's Freestyle...|   Y|Doesn't work that...|Elmer’s is a trus...|
|   50579471|      42258955|V-LIGHT Adjustabl...|   Y|             Sturdy!|I really like thi...|
|   48428870|      12471266|Avery White 4-1/4...|   Y|Perfect for makin...|I was really plea...|
|   17217532|     467158061|Av

In [ ]:
#we are doen to 6 columns from our original 15. The vine column is our primary tool for analysis, we have 5 columns left to review. We can tokenize 3 out of the five columns.

In [ ]:
product_title_df = df.select("product_title", "vine")

In [ ]:
product_title_Yes_df = product_title_df.filter(product_title_df["vine"] == "Y")

In [ ]:
product_title_Yes_df.show()

+--------------------+----+
|       product_title|vine|
+--------------------+----+
|Westcott iPoint O...|   Y|
|V-LIGHT LED Energ...|   Y|
|Elmer's Freestyle...|   Y|
|Boxa HOPPER Elega...|   Y|
|Elmer's Freestyle...|   Y|
|V-LIGHT Adjustabl...|   Y|
|Avery White 4-1/4...|   Y|
|Avery Ultralast B...|   Y|
|Avery Easy Align ...|   Y|
|Five Star 3 Subje...|   Y|
|Scotch Thermal La...|   Y|
|Avery Ultralast B...|   Y|
|Westcott iPoint O...|   Y|
|Studio Ink Assort...|   Y|
|Avery White Inter...|   Y|
|Five Star Locker ...|   Y|
|Elmer's Freestyle...|   Y|
|Five Star Advance...|   Y|
|Five Star Extra T...|   Y|
|Five Star 3 Subje...|   Y|
+--------------------+----+
only showing top 20 rows



In [ ]:
review_headline_df = df.select("review_headline", "vine")

In [ ]:
review_headline_Yes_df = review_headline_df.filter(review_headline_df["vine"] == "Y")

In [ ]:
review_headline_Yes_df.show()

+--------------------+----+
|     review_headline|vine|
+--------------------+----+
|Kid proof, or as ...|   Y|
|... V-Light LED E...|   Y|
|              Useful|   Y|
|Not exciting, but...|   Y|
|Doesn't work that...|   Y|
|             Sturdy!|   Y|
|Perfect for makin...|   Y|
|Love these for at...|   Y|
|Actual, self-lami...|   Y|
|Five Star Vertica...|   Y|
|For occasional la...|   Y|
|Quality is also g...|   Y|
|Eletric pencil sh...|   Y|
|The cards are oka...|   Y|
|Perfect for use i...|   Y|
|  Daughter Loves It!|   Y|
|  Didn't work for me|   Y|
|Great for persona...|   Y|
|Stands tall on it...|   Y|
|Quite possibly th...|   Y|
+--------------------+----+
only showing top 20 rows



In [ ]:
review_body_df = df.select("review_body", "vine")

In [ ]:
review_body_Yes_df = review_body_df.filter(review_body_df["vine"] == "Y")

In [ ]:
review_body_Yes_df.show()

+--------------------+----+
|         review_body|vine|
+--------------------+----+
|It seems electric...|   Y|
|The V-Light LED E...|   Y|
|I use this light ...|   Y|
|This is a review ...|   Y|
|Elmer’s is a trus...|   Y|
|I really like thi...|   Y|
|I was really plea...|   Y|
|Whether or not yo...|   Y|
|These are much mo...|   Y|
|I don’t know that...|   Y|
|For a low-budget ...|   Y|
|I had the chance ...|   Y|
|Amazon sent me tw...|   Y|
|When I buy someth...|   Y|
|I don't actually ...|   Y|
|This product prob...|   Y|
|I tried attaching...|   Y|
|I haven't had a s...|   Y|
|Excellent product...|   Y|
|Although this pro...|   Y|
+--------------------+----+
only showing top 20 rows



In [ ]:
# Import the Tokenizer library.
from pyspark.ml.feature import Tokenizer

In [ ]:
# Create a function to return the length of a list
def word_list_length(word_list):
	return len(word_list)

In [ ]:

from pyspark.sql.functions import col, udf
from pyspark.sql.types import IntegerType

In [ ]:

# Create a user defined function
count_tokens = udf(word_list_length, IntegerType())

In [ ]:
# Create our Tokenize sentences
tokenizer = Tokenizer(inputCol="product_title", outputCol="words")

In [ ]:
# Transform DataFrame
product_title_tokenized = tokenizer.transform(product_title_Yes_df)

In [ ]:
# Select the needed columns and no truncated results
product_title_tokenized.withColumn("tokens", count_tokens(col("words"))).show(truncate=False)

+-------------------------------------------------------------------------------------------------------------------------------------------+----+-------------------------------------------------------------------------------------------------------------------------------------------------------------------+------+
|product_title                                                                                                                              |vine|words                                                                                                                                                              |tokens|
+-------------------------------------------------------------------------------------------------------------------------------------------+----+-------------------------------------------------------------------------------------------------------------------------------------------------------------------+------+
|Westcott iPoint Orbit Electric Pencil Sharpen

In [ ]:
# Create the Tokenized sentences
tokenizer = Tokenizer(inputCol="review_headline", outputCol="words")

In [ ]:
# Transform DataFrame
review_headline_tokenized = tokenizer.transform(review_headline_Yes_df)

In [ ]:
# Select the needed columns and don't truncate results
review_headline_tokenized.withColumn("tokens", count_tokens(col("words"))).show(truncate=False)

+----------------------------------------------------------------------------------------------+----+---------------------------------------------------------------------------------------------------------------+------+
|review_headline                                                                               |vine|words                                                                                                          |tokens|
+----------------------------------------------------------------------------------------------+----+---------------------------------------------------------------------------------------------------------------+------+
|Kid proof, or as close as it gets.                                                            |Y   |[kid, proof,, or, as, close, as, it, gets.]                                                                    |8     |
|... V-Light LED Energy Efficient Pyramid Style Lamp is the perfect size for my small secretary|Y   |[..., v-light, 

In [ ]:
# Create our Tokenize sentences
tokenizer = Tokenizer(inputCol="review_body", outputCol="words")

In [ ]:
# Transform DataFrame
review_body_tokenized = tokenizer.transform(review_body_Yes_df)

In [ ]:
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://dataviz2.cdvvuej8kzy3.us-east-2.rds.amazonaws.com:5432/Challenge"
config = {"user":"postgres2", 
          "password": "passenger8808", 
          "driver":"org.postgresql.Driver"}

In [ ]:
# Write DataFrame to office_user table in RDS
office_df.write.jdbc(url=jdbc_url, table='office', mode=mode, properties=config)

In [ ]:
# Write DataFrame to office_user table in RDS
product_title_Yes_df.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

In [ ]:
# Write DataFrame to office_user table in RDS
review_body_Yes_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)